In [12]:
import jax
import jax.numpy as jnp
import pandas as pd
import plotly.express as px

from jax import random, lax, jit, vmap, pmap
from tqdm.auto import tqdm
from functools import partial

from policies import BasePolicy
from agents import Q_learning
from envs import GridWorld
from policies import EpsilonGreedy

In [13]:
SEED = 2
GRID_DIM = jnp.array([8, 8])
INITIAL_STATE = jnp.array([8, 8])
GOAL_STATE = jnp.array([2, 0])
GRID_SIZE = jnp.array([8, 8])
N_STATES = jnp.prod(GRID_DIM)
N_ACTIONS = 4
DISCOUNT = 0.9
LEARNING_RATE = 0.1

key = random.PRNGKey(SEED)

env = GridWorld(INITIAL_STATE, GOAL_STATE, GRID_SIZE)
policy = EpsilonGreedy(0.1)
agent = Q_learning(
    key,
    N_STATES,
    N_ACTIONS,
    DISCOUNT,
    LEARNING_RATE,
    policy,
)

movements = jnp.array([[0, 1], [1, 0], [0, -1], [-1, 0]])


In [14]:
def train_n_steps(n_steps=1000):    
    
    key = random.PRNGKey(0)
    q_values = jnp.zeros([GRID_SIZE[0], GRID_SIZE[1], N_ACTIONS], dtype=jnp.float32)
    env_state, obs = env.reset(key) 
    done = False

    steps = []

    for _ in tqdm(range(n_steps)):
        done = False
        n_steps = 0

        while not done:
            state, _ = env_state
            action, key = policy(key, N_ACTIONS, state, q_values)
            movement = movements[action]
            env_state, obs, reward, done = env.step(env_state, movement)

            q_values = agent.update(state, action, reward, done, obs, q_values)
            n_steps+=1
        steps.append(n_steps)


    fig=px.imshow(pd.DataFrame(jnp.max(q_values, axis=2)).round(2), title="Maximal Q-value for each state")
    fig.show()
    fig=px.line(steps)
    fig.show()
    
# train_n_steps()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:20<00:00, 48.42it/s]


In [16]:
v_reset = jit(
            vmap(
                env.reset,
                out_axes=((0, 0), 0),  # ((env_state), obs)
                axis_name="batch_axis",
            )
        )

v_step = jit(
            vmap(
                env.step,
                in_axes=((0, 0), 0),  # ((env_state), action)
                out_axes=((0, 0), 0, 0, 0),  # ((env_state), obs, reward, done)
                axis_name="batch_axis",
            )
        )
v_update = jit(
        vmap(
            agent.update,
            in_axes=(0, 0, 0, 0, 0, -1),
            axis_name="batch_axis",
        ),
    )

v_policy = jit(
    vmap(
        policy.call,
        in_axes=(0, None, 0, -1),  # (keys, n_actions, state, q_values)
        axis_name="batch_axis",
    ),
    static_argnums=(1,),
)

In [20]:
N_ENV = 10

key = random.PRNGKey(SEED)
keys = random.split(key, N_ENV)
action_keys = random.split(random.PRNGKey(0), N_ENV) 


env_states, obs = v_reset(keys)
states, keys = env_states
q_values = jnp.zeros([GRID_SIZE[0], GRID_SIZE[1], N_ACTIONS, N_ENV], dtype=jnp.float32)

actions, action_keys = v_policy(action_keys, N_ACTIONS, states, q_values)

env_state, obs, reward, done = v_step(env_states, actions)
states, keys = env_state

q_values = v_update(states, actions, reward, done, obs, q_values)

In [29]:
@pmap
@vmap
def rollout(keys, timesteps, env):
    all_obs = jnp.zeros([timesteps, N_ENV])
    all_rewards = jnp.zeros([timesteps, N_ENV], dtype=jnp.int32)
    all_done = jnp.zeros([timesteps, N_ENV], dtype=jnp.bool)
    all_q_values = jnp.zeros([timesteps, GRID_SIZE[0], GRID_SIZE[1], N_ACTIONS, N_ENV], dtype=jnp.float32)

    action_keys = random.PRNGKey(0, N_ENV)
    env_states, obs = v_reset(keys)
    
    val = (env_states, action_keys, all_obs, all_rewards, all_done, all_q_values)
    val = lax.fori_loop(0, timesteps, fori_body, val)
    env_states, action_keys, all_obs, all_reward, all_done, all_q_values = val
    return all_obs, all_reward, all_done, all_q_values

In [30]:
def fori_body(i:int, val:tuple):
    env_states, action_key, all_obs, all_rewards, all_done, all_q_values = val
    states, keys = env_states
    actions, action_keys = v_policy(action_keys, N_ACTIONS, states, q_values)
    
    env_states, obs, rewards, done = v_step(env_states, actions)
    q_values = v_update(states, actions, rewards, done, obs, q_values[i, :, :, :, :])

    all_obs = all_obs.at[i].set(obs)
    all_rewards = all_rewards.at[i].set(rewards)
    all_done = all_done.at[i].set(done)
    all_q_values = all_q_values.at[i].set(q_values)

    val = (env_states, action_keys, all_obs, all_rewards, all_done, all_q_values)
    return val


In [32]:
key = random.PRNGKey(SEED)
keys = random.split(key, N_ENV)
rollout(keys, 10, env)

ValueError: pmap was requested to map its argument along axis 0, which implies that its rank should be at least 1, but is only 0 (its shape is ())